In [1]:
!pip install evaluate

In [2]:
import torch
import pandas as pd
import numpy as np
import nltk
import os
import optuna
from nltk.tokenize import word_tokenize, sent_tokenize
import fasttext
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from gensim.models import FastText
import re
import transformers
import spacy
from nltk.stem import PorterStemmer, SnowballStemmer
from datasets import load_dataset
import evaluate
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from scipy.special import softmax
from catboost import CatBoostClassifier, Pool
import warnings
warnings.filterwarnings('ignore')

2024-05-03 17:53:00.815324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 17:53:00.815385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 17:53:00.816947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
train = pd.read_csv('/kaggle/input/vk-spam-text/train_spam.csv')
test = pd.read_csv('/kaggle/input/vk-spam-text/test_spam.csv')

In [5]:
nlp = spacy.load('en_core_web_lg')
ss = SnowballStemmer(language='english')

In [6]:
train

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
...,...,...
16273,spam,if you are interested in binary options tradin...
16274,spam,dirty pictureblyk on aircel thanks you for bei...
16275,ham,or you could do this g on mon 1635465 sep 1635...
16276,ham,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [7]:
#Баланс классов
train['text_type'].value_counts(normalize=True)

text_type
ham     0.704571
spam    0.295429
Name: proportion, dtype: float64

## Препроцессинг текста, после нескольких экспериментов лучшим подходом оказался:
* Приведение к нижнему регистру
* Удаление пунктуации
* Удаление служебных знаков
* Удаление стоп слов
* Стемминг

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктцацию
#     text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = text.replace("\n"," ")
    text = re.sub('\s{2,}', " ", text)
    text = remove_stopwords(text)
    text = ' '.join(list(map(ss.stem, word_tokenize(text)))) # Стемминг
#     text = ' '.join([token.lemma_ for token in nlp(text)]) # Лемматизация
    return text

In [9]:
train['text'] = train['text'].apply(clean_text)

In [10]:
test['text'] = test['text'].apply(clean_text)

In [11]:
train

,text_type,text
0,ham,sure alex know birthday minut far your concern
1,ham,resum john lavorato thank vinc move right away...
2,spam,plzz visit websit moviesgodml movi free provid...
3,spam,urgent mobil number award 2000 prize guarante ...
4,ham,overview hr associ analyst project david s req...
...,...,...
16273,spam,interest binari option trade continu inform dm...
16274,spam,dirti pictureblyk aircel thank valu member her...
16275,ham,g mon 1635465 sep 1635465 david ree wrote mon ...
16276,ham,insta reel par 80 bhara pada hai kuch bhi dalt...


In [12]:
# Замена строчных классов на 0 и 1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['text_type'] = le.fit_transform(train['text_type'])

In [13]:
train_df, valid_df = train_test_split(train, test_size = 0.2, random_state=0)

In [14]:
train_df

,text_type,text
3959,1,noob sneha sneha
14899,0,parti place usf charg contribut way great appr...
16136,1,grand slam stox momentum alert issu juli 6 8 2...
13931,0,jedi share vinc attach descript deal involv je...
12487,1,sub kenya uga sa philipin ng usa tz jamaica zi...
...,...,...
13123,0,msng u lot dher saari baatein batani h gn jaan...
3264,0,swt thought nver tire littl thing 4 lovabl per...
9845,1,passiv incom key financi freedom time know t a...
10799,0,whos guilti wife dreamng d night suden shout q...


In [15]:
with open('train.txt', 'w') as f:
    for each_text, each_label in zip(train_df['text'], train_df['text_type']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [16]:
with open('test.txt', 'w') as f:
    for each_text in zip(valid_df['text'], valid_df['text_type']):
        f.writelines(f'__label__{each_label} {each_text}\n')

# Модель для классификации из библиотеки fasttext

In [17]:
model = fasttext.train_supervised('train.txt', wordNgrams = 2,epoch=75,lr=0.35)

def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

print_results(*model.test('test.txt'))

Read 0M words
Number of words:  33725
Number of labels: 2
Progress: 100.0% words/sec/thread: 1088346 lr:  0.000000 avg.loss:  0.070201 ETA:   0h 0m 0s


sample_size=3256
precision=0.6
recall=0.6


In [18]:
y_pred_proba = model.predict(valid_df['text'].to_list())

y_pred = []
for clses, prob in zip(*y_pred_proba):
    clss = int(clses[0].replace("__label__",""))
    if clss == 1:
        y_pred.append(prob[0])
    else:
        y_pred.append(1-prob[0])
        
        

# Расчет ROC AUC
roc_auc = roc_auc_score(valid_df['text_type'].to_list(), y_pred)

print("ROC AUC:", roc_auc)

ROC AUC: 0.9690817101485899


In [19]:
y_pred_proba = model.predict(test['text'].to_list())

y_pred = []
for clses, prob in zip(*y_pred_proba):
    clss = int(clses[0].replace("__label__",""))
    if clss == 1:
        y_pred.append(prob[0])
    else:
        y_pred.append(1-prob[0])
        
        

submit = pd.DataFrame({'text' : test['text'].to_list(), 'score': np.abs(np.round(y_pred,4))})
submit.to_csv('submission_ft.csv')

In [20]:
submit.describe()

,score
count,4070.000000
mean,0.324216
std,0.440185
min,0.000000
25%,0.000000
50%,0.002900
75%,0.959375
max,1.000000


# Fine Tuning DisitBERT из библиотеки transformers

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [22]:
train_enc = tokenizer(train_df['text'].astype(str).to_list(), truncation=True)
valid_enc = tokenizer(valid_df['text'].astype(str).to_list(), truncation=True)


In [23]:
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDataset(train_enc, train_df['text_type'].to_list())
val_dataset = SpamDataset(valid_enc, valid_df['text_type'].to_list())

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
import evaluate
rocauc = evaluate.load("roc_auc")

In [26]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return rocauc.compute(prediction_scores=predictions, references=labels)

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: yakto. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Roc Auc
1,No log,0.175083,0.934800
2,No log,0.129654,0.947685
3,No log,0.137145,0.946211
4,No log,0.168750,0.930914
5,0.135200,0.153227,0.942483
6,0.135200,0.166958,0.938080
7,0.135200,0.156014,0.952815
8,0.135200,0.155295,0.956398
9,0.135200,0.172626,0.947374
10,0.025800,0.167765,0.953085


TrainOutput(global_step=1020, training_loss=0.07925674848112406, metrics={'train_runtime': 1244.2595, 'train_samples_per_second': 104.657, 'train_steps_per_second': 0.82, 'total_flos': 7157678737631304.0, 'train_loss': 0.07925674848112406, 'epoch': 10.0})

# Эмбеддинг FastText из gensim + catboost

In [29]:
model_ft = FastText(vector_size=400, window=5, min_count=3,seed=0)
model_ft.build_vocab(corpus_iterable=train_df['text'].to_list())
total_words = model_ft.corpus_total_words
model_ft.train(corpus_iterable=train_df['text'].to_list(),
            total_examples=len(train_df['text'].to_list()),
            total_words=total_words,
            epochs=20,)

(10022975, 49849900)

In [30]:
def vectorize_sum(comment, embeddings):
    embedding_dim = embeddings.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')

    for word in word_tokenize(comment):
        features += embeddings[f'{word}']
    
    return features

In [31]:
train_ft = np.stack([vectorize_sum(text, model_ft.wv) for text in train_df['text'].to_list()])
valid_ft = np.stack([vectorize_sum(text, model_ft.wv) for text in valid_df['text'].to_list()])

In [32]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=100000,random_state=0)
wv_model = clf.fit(train_ft, train_df['text_type'].to_list())
roc_auc_score(valid_df['text_type'].to_list(), wv_model.predict_proba(valid_ft)[:,1])

0.7901179217619746

In [33]:
train_label = train_df['text_type'].to_list()
valid_label = valid_df['text_type'].to_list()

In [36]:
def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 12),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    model.fit(train_ft, train_label)
    y_pred = model.predict_proba(valid_ft)[:,1]
    return roc_auc_score(valid_label, y_pred)

opt_cat = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=6), direction='maximize')
opt_cat.optimize(objective, n_trials=30, timeout=3000,show_progress_bar=True)
opt_cat_params = opt_cat.best_params

[I 2024-05-03 18:18:58,922] A new study created in memory with name: no-name-e8f44e55-b7c5-4189-99da-faf0d812c415


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2024-05-03 18:20:54,423] Trial 0 finished with value: 0.8583131921206761 and parameters: {'iterations': 342, 'learning_rate': 0.0745715230201093, 'depth': 9, 'l2_leaf_reg': 0.006771220582203362, 'random_strength': 6.852858627079942e-05, 'bagging_temperature': 7.487914451463723, 'od_wait': 20}. Best is trial 0 with value: 0.8583131921206761.
[I 2024-05-03 18:21:23,839] Trial 1 finished with value: 0.7895783319951145 and parameters: {'iterations': 890, 'learning_rate': 0.003195902773186976, 'depth': 4, 'l2_leaf_reg': 0.0002738606439687508, 'random_strength': 1.147497025593285, 'bagging_temperature': 7.331181631031485, 'od_wait': 21}. Best is trial 0 with value: 0.8583131921206761.
[I 2024-05-03 18:22:46,479] Trial 2 finished with value: 0.8129394214818769 and parameters: {'iterations': 754, 'learning_rate': 0.0029707942747752764, 'depth': 7, 'l2_leaf_reg': 2.6659427482203685e-07, 'random_strength': 1.9693067655472611, 'bagging_temperature': 2.7625571379675993, 'od_wait': 14}. Best is 

In [37]:
model_cat = CatBoostClassifier(**opt_cat_params, verbose=100,random_seed=0)
model_cat.fit(train_ft, train_label)
y_pred = model_cat.predict_proba(valid_ft)[:,1]
roc_auc_score(valid_label, y_pred)

0:	learn: 0.6568641	total: 508ms	remaining: 1m 17s
100:	learn: 0.2856954	total: 52.6s	remaining: 27.6s
153:	learn: 0.2428384	total: 1m 20s	remaining: 0us


0.869397991720705

# Вывод: Лучшим подходом оказалась модель из библиотеки fasttext, которая показала лучшую метрику roc-auc и время обучения